In [19]:
pip install pandas numpy scikit-learn tqdm torch geneticalgorithm matplotlib optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.9/603.9 kB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 45.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
# ---------------------- Importing Packages ---------------------- #
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import optuna
import random

# ---------------------- Reproducibility ---------------------- #
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

# Select CPU or GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------------- Fold Configuration Settings ---------------------- #
val_window_num_sequences = 252
holdout_base = 756
forecast_horizons = [1, 5, 21, 63, 252]
num_epochs = 50
patience = 10

# ---------------------- LSTM Classifier ---------------------- #
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout=0.0):
        super().__init__()
        effective_dropout = dropout if num_layers > 1 else 0.0
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=effective_dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        return self.fc(hn[-1])

# ---------------------- Helper: Sequence Creation ---------------------- #
def create_sequences(X_df, Y_df, seq_len, forecast_horizon):
    X_df = X_df.dropna()
    Y_df = Y_df.dropna()
    common_idx = X_df.index.intersection(Y_df.index)
    X_df = X_df.loc[common_idx]
    Y_df = Y_df.loc[common_idx]

    X_arr = X_df.values.astype(np.float32)
    Y_arr = Y_df.values.astype(np.float32).reshape(-1, 1)

    X_seq, Y_seq = [], []
    max_i = len(X_arr) - seq_len - forecast_horizon + 1
    for i in range(max_i):
        X_seq.append(X_arr[i:i + seq_len])
        Y_seq.append(Y_arr[i + seq_len + forecast_horizon - 1])

    X_seq = np.array(X_seq)
    Y_seq = np.array(Y_seq)
    return X_seq, Y_seq

# ---------------------- Feature Shifting ---------------------- #
def shift_X_by_horizon(X_df, horizon):
    return X_df.shift(horizon).dropna()

# ---------------------- Standardization ---------------------- #
def standardize_fold(X_train, X_val):
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)
    X_val_scaled = pd.DataFrame(scaler.transform(X_val), index=X_val.index, columns=X_val.columns)
    return X_train_scaled, X_val_scaled

# ---------------------- Generate Expanding Folds ---------------------- #
def get_expanding_folds(X_df, Y_df, forecast_horizon, sequence_length, val_window_num_sequences, holdout_base):
    assert X_df.index.equals(Y_df.index)
    total_days = len(X_df)
    val_window = sequence_length + forecast_horizon + val_window_num_sequences - 1
    min_train_window = sequence_length + forecast_horizon + val_window_num_sequences - 1

    fold_starts = []
    i = min_train_window
    while i + val_window + holdout_base <= total_days:
        fold_starts.append(i)
        i += val_window

    folds = []
    for fold_start in fold_starts:
        train_end = fold_start
        val_start = fold_start
        val_end = fold_start + val_window

        X_train = X_df.iloc[:train_end].copy()
        Y_train = Y_df.iloc[:train_end].copy()
        X_val = X_df.iloc[val_start:val_end].copy()
        Y_val = Y_df.iloc[val_start:val_end].copy()

        folds.append({
            "X_train": X_train,
            "Y_train": Y_train,
            "X_val": X_val,
            "Y_val": Y_val,
            "fold_start": fold_start,
            "fold_end": val_end,
        })

    last_val_end = folds[-1]['fold_end']
    holdout_days = total_days - last_val_end
    print(f"[INFO] Generated {len(folds)} folds for forecast horizon {forecast_horizon}")
    return folds, last_val_end, holdout_days

# ---------------------- Optuna Objective (Sequence + Feature Selection) ---------------------- #
def optuna_objective(trial, X_df, Y_df, forecast_horizon):
    sequence_len = trial.suggest_int("sequence_length", 756, 1512, step=63)

    # Feature selection
    all_columns = list(X_df.columns)
    selected_cols = [col for col in all_columns if trial.suggest_categorical(f"use_{col}", [True, False])]
    if not selected_cols:
        return float('inf')  # skip trials with no features selected

    X_filtered = X_df[selected_cols]
    folds, _, _ = get_expanding_folds(X_filtered, Y_df, forecast_horizon, sequence_len, val_window_num_sequences, holdout_base)

    total_f1 = 0
    for fold in folds:
        X_train_raw, X_val_raw = fold["X_train"], fold["X_val"]
        Y_train_raw, Y_val_raw = fold["Y_train"], fold["Y_val"]
        X_train_std, X_val_std = standardize_fold(X_train_raw, X_val_raw)

        X_train_seq, Y_train_seq = create_sequences(X_train_std, Y_train_raw, sequence_len, forecast_horizon)
        X_val_seq, Y_val_seq = create_sequences(X_val_std, Y_val_raw, sequence_len, forecast_horizon)

        if len(X_train_seq) == 0 or len(X_val_seq) == 0:
            continue

        X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
        Y_train_tensor = torch.tensor(Y_train_seq, dtype=torch.float32).view(-1, 1).to(device)
        X_val_tensor = torch.tensor(X_val_seq, dtype=torch.float32).to(device)
        Y_val_tensor = torch.tensor(Y_val_seq, dtype=torch.float32).view(-1, 1).to(device)

        model = LSTMClassifier(input_dim=X_train_seq.shape[2], hidden_dim=64, num_layers=1, output_dim=1, dropout=0.2).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        pos_weight_val = (Y_train_tensor == 0).sum() / (Y_train_tensor == 1).sum()
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_val)

        train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=64, shuffle=True)
        val_loader = DataLoader(TensorDataset(X_val_tensor, Y_val_tensor), batch_size=64)

        best_f1 = 0
        patience_counter = 0
        for epoch in range(num_epochs):
            model.train()
            for xb, yb in train_loader:
                optimizer.zero_grad()
                loss = criterion(model(xb), yb)
                loss.backward()
                optimizer.step()

            model.eval()
            preds = []
            with torch.no_grad():
                for xb, _ in val_loader:
                    preds.append(torch.sigmoid(model(xb)))
            pred_tensor = torch.cat(preds, dim=0).squeeze()
            pred_class = (pred_tensor > 0.5).int()
            f1 = f1_score(Y_val_tensor.cpu(), pred_class.cpu())

            if f1 > best_f1:
                best_f1 = f1
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    break

        total_f1 += best_f1

    return -total_f1 / len(folds)

# ---------------------- Run Optuna ---------------------- #
def run_optuna_optimization(X_df, Y_df, forecast_horizon, n_trials=30):
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: optuna_objective(trial, X_df, Y_df, forecast_horizon), n_trials=n_trials)
    return study.best_params, -study.best_value

# ---------------------- Run All Horizons ---------------------- #
def run_for_all_horizons(X_df, Y_df_dict):
    all_results = {}
    for h in forecast_horizons:
        print(f"\n=== Forecast Horizon: {h} ===")
        Y_df = Y_df_dict[h]
        X_shifted = shift_X_by_horizon(X_df, h)
        Y_aligned = Y_df.loc[X_shifted.index]
        X_final, Y_final = X_shifted, Y_aligned
        best_params, best_f1 = run_optuna_optimization(X_final, Y_final, h)
        all_results[h] = {"best_params": best_params, "best_f1": best_f1}
        print(f"[RESULT] Horizon {h}: Best Params = {best_params}, Best F1 = {best_f1:.4f}")
    return all_results


In [4]:
# Load data
X_df = pd.read_csv(r"X_df_filtered.csv", index_col=0, parse_dates=True)
Y_df_dict = {
    1: pd.read_csv(r"Y_df_change_dir_1.csv", index_col=0, parse_dates=True),
    5: pd.read_csv(r"Y_df_change_dir_5.csv", index_col=0, parse_dates=True),
    21: pd.read_csv(r"Y_df_change_dir_21.csv", index_col=0, parse_dates=True),
    63: pd.read_csv(r"Y_df_change_dir_63.csv", index_col=0, parse_dates=True),
    252: pd.read_csv(r"Y_df_change_dir_252.csv", index_col=0, parse_dates=True),
}

# Run optimization
results = run_for_all_horizons(X_df, Y_df_dict)

[I 2025-05-11 16:01:16,729] A new study created in memory with name: no-name-7a68c724-4cf0-4090-9a5b-27c6cb6da5cb



=== Forecast Horizon: 1 ===
[INFO] Generated 2 folds for forecast horizon 1


[I 2025-05-11 16:01:18,624] Trial 0 finished with value: -0.2467948717948718 and parameters: {'sequence_length': 1071, 'use_EUDR1T  Index': False, 'use_EUR009M  Index': True, 'use_ER1  Index': True, 'use_ER2  Index': True, 'use_ER3  Index': True, 'use_ER4  Index': True, 'use_ER5  Index': True, 'use_FDTR  Index': True, 'use_GJTB3MO  Index': False, 'use_DU1  Index': False, 'use_RX1  Index': True, 'use_SX5E  Index': False, 'use_SX7E  Index': False, 'use_HSI  Index': True, 'use_EURUSD   Index': False, 'use_EURJPY   Index': False, 'use_EURGBP   Index': False, 'use_EURCHF   Index': False, 'use_CRY  Index': False, 'use_C 1  Index': False, 'use_NG1  Index': False, 'use_MOVE  Index': True, 'use_VIX  Index': False, 'use_GDP CQOQ Index': False, 'use_CPI XYOY  Index': True, 'use_NAPMPMI  Index': True, 'use_IP CHNG  Index': False, 'use_CPTICHNG  Index': True, 'use_USURTOT  Index': False, 'use_NFP TCH  Index': True, 'use_INJCJC  Index': True, 'use_LEI CHNG  Index': False, 'use_EUGNEMUY  Index': Fals

[INFO] Generated 2 folds for forecast horizon 1


[I 2025-05-11 16:01:21,722] Trial 1 finished with value: -0.460993731135361 and parameters: {'sequence_length': 1260, 'use_EUDR1T  Index': True, 'use_EUR009M  Index': True, 'use_ER1  Index': True, 'use_ER2  Index': False, 'use_ER3  Index': True, 'use_ER4  Index': True, 'use_ER5  Index': False, 'use_FDTR  Index': False, 'use_GJTB3MO  Index': True, 'use_DU1  Index': False, 'use_RX1  Index': True, 'use_SX5E  Index': False, 'use_SX7E  Index': False, 'use_HSI  Index': True, 'use_EURUSD   Index': True, 'use_EURJPY   Index': True, 'use_EURGBP   Index': False, 'use_EURCHF   Index': False, 'use_CRY  Index': True, 'use_C 1  Index': True, 'use_NG1  Index': False, 'use_MOVE  Index': False, 'use_VIX  Index': False, 'use_GDP CQOQ Index': False, 'use_CPI XYOY  Index': True, 'use_NAPMPMI  Index': True, 'use_IP CHNG  Index': False, 'use_CPTICHNG  Index': True, 'use_USURTOT  Index': False, 'use_NFP TCH  Index': True, 'use_INJCJC  Index': True, 'use_LEI CHNG  Index': True, 'use_EUGNEMUY  Index': False, '

...

[INFO] Generated 1 folds for forecast horizon 252


[I 2025-05-11 16:04:55,063] Trial 28 finished with value: -0.6976744186046512 and parameters: {'sequence_length': 1323, 'use_EUDR1T  Index': True, 'use_EUR009M  Index': False, 'use_ER1  Index': False, 'use_ER2  Index': True, 'use_ER3  Index': True, 'use_ER4  Index': True, 'use_ER5  Index': True, 'use_FDTR  Index': True, 'use_GJTB3MO  Index': False, 'use_DU1  Index': True, 'use_RX1  Index': False, 'use_SX5E  Index': True, 'use_SX7E  Index': True, 'use_HSI  Index': True, 'use_EURUSD   Index': False, 'use_EURJPY   Index': True, 'use_EURGBP   Index': False, 'use_EURCHF   Index': True, 'use_CRY  Index': False, 'use_C 1  Index': True, 'use_NG1  Index': False, 'use_MOVE  Index': False, 'use_VIX  Index': False, 'use_GDP CQOQ Index': True, 'use_CPI XYOY  Index': True, 'use_NAPMPMI  Index': False, 'use_IP CHNG  Index': False, 'use_CPTICHNG  Index': True, 'use_USURTOT  Index': False, 'use_NFP TCH  Index': False, 'use_INJCJC  Index': True, 'use_LEI CHNG  Index': True, 'use_EUGNEMUY  Index': True, 

[INFO] Generated 1 folds for forecast horizon 252


/venv/main/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/venv/main/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/venv/main/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/venv/main/lib/python3.12/site-packages/sklearn/metr

[RESULT] Horizon 252: Best Params = {'sequence_length': 1323, 'use_EUDR1T  Index': True, 'use_EUR009M  Index': False, 'use_ER1  Index': False, 'use_ER2  Index': True, 'use_ER3  Index': True, 'use_ER4  Index': True, 'use_ER5  Index': True, 'use_FDTR  Index': True, 'use_GJTB3MO  Index': False, 'use_DU1  Index': True, 'use_RX1  Index': False, 'use_SX5E  Index': True, 'use_SX7E  Index': True, 'use_HSI  Index': True, 'use_EURUSD   Index': False, 'use_EURJPY   Index': True, 'use_EURGBP   Index': False, 'use_EURCHF   Index': True, 'use_CRY  Index': False, 'use_C 1  Index': True, 'use_NG1  Index': False, 'use_MOVE  Index': False, 'use_VIX  Index': False, 'use_GDP CQOQ Index': True, 'use_CPI XYOY  Index': True, 'use_NAPMPMI  Index': False, 'use_IP CHNG  Index': False, 'use_CPTICHNG  Index': True, 'use_USURTOT  Index': False, 'use_NFP TCH  Index': False, 'use_INJCJC  Index': True, 'use_LEI CHNG  Index': True, 'use_EUGNEMUY  Index': True, 'use_CPEXEMUY  Index': False, 'use_EUIPEMUY  Index': False

In [ ]:
# ---------------------- Fold Debugging Script ---------------------- #
# Load feature data
X_df = pd.read_csv(r'C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Data Processing\Output\Independent\X_df.csv', index_col=0, parse_dates=True)
X_df.index = pd.to_datetime(X_df.index)

# Configs
sequence_length = 252
holdout_base = 756
forecast_horizons = [1, 5, 21, 63, 252]

# Run diagnostics for each forecast horizon
for h in forecast_horizons:
    print(f"\n=== Forecast Horizon: {h} ===")

    # Load directional labels for the current horizon
    Y_df = pd.read_csv(fr'C:\Users\azorb\PycharmProjects\Predicting the Yield Curve\Data Processing\Output\Dependent\Classification\Y_df_change_dir_{h}.csv', index_col=0, parse_dates=True)
    Y_df.index = pd.to_datetime(Y_df.index)

    # Shift and align
    X_shifted = shift_X_by_horizon(X_df, h)
    common_idx = X_shifted.index.intersection(Y_df.index)
    X_aligned = X_shifted.loc[common_idx]
    Y_aligned = Y_df.loc[common_idx]

    assert X_aligned.index.equals(Y_aligned.index), "Index mismatch after aligning!"

    # Generate folds
    val_window = val_window = sequence_length + h + 252 - 1  # or 1008 + forecast_horizon
    folds, last_val_end, holdout_days = get_expanding_folds(X_aligned, Y_aligned, h, sequence_length, val_window, holdout_base)

    # First fold shapes
    X_train, Y_train = folds[0]["X_train"], folds[0]["Y_train"]
    X_val, Y_val = folds[0]["X_val"], folds[0]["Y_val"]
    print(f"[DEBUG] First fold train X: {X_train.shape}, Y: {Y_train.shape}")
    print(f"[DEBUG] First fold val X: {X_val.shape}, Y: {Y_val.shape}")

    # Standardize
    X_train_std, X_val_std = standardize_fold(X_train, X_val)

    # Sequences
    X_train_seq, Y_train_seq = create_sequences(X_train_std, Y_train, sequence_length, h)
    X_val_seq, Y_val_seq = create_sequences(X_val_std, Y_val, sequence_length, h)

    print(f"[DEBUG] Train sequences X: {X_train_seq.shape}, Y: {Y_train_seq.shape}")
    print(f"[DEBUG] Val sequences X: {X_val_seq.shape}, Y: {Y_val_seq.shape}")

    # Check correspondence of sequence and target alignment
    if len(X_train_seq) > 0:
        print(f"[INFO] First target index: {Y_train.index[sequence_length + h - 1]}")
        print(f"[INFO] Expected Y index from X: {X_train.index[0]} to {X_train.index[sequence_length - 1]}")

In [5]:
results

{1: {'best_params': {'sequence_length': 945,
   'use_EUDR1T  Index': False,
   'use_EUR009M  Index': False,
   'use_ER1  Index': True,
   'use_ER2  Index': True,
   'use_ER3  Index': False,
   'use_ER4  Index': True,
   'use_ER5  Index': False,
   'use_FDTR  Index': False,
   'use_GJTB3MO  Index': False,
   'use_DU1  Index': True,
   'use_RX1  Index': True,
   'use_SX5E  Index': True,
   'use_SX7E  Index': True,
   'use_HSI  Index': False,
   'use_EURUSD   Index': False,
   'use_EURJPY   Index': False,
   'use_EURGBP   Index': True,
   'use_EURCHF   Index': True,
   'use_CRY  Index': True,
   'use_C 1  Index': True,
   'use_NG1  Index': True,
   'use_MOVE  Index': False,
   'use_VIX  Index': True,
   'use_GDP CQOQ Index': True,
   'use_CPI XYOY  Index': False,
   'use_NAPMPMI  Index': False,
   'use_IP CHNG  Index': False,
   'use_CPTICHNG  Index': False,
   'use_USURTOT  Index': False,
   'use_NFP TCH  Index': True,
   'use_INJCJC  Index': False,
   'use_LEI CHNG  Index': False,
   'u

In [31]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import warnings

# ---------------------- Evaluate on Holdout Test Set ---------------------- #
def evaluate_on_holdout(X_df, Y_df_dict, best_params_dict):
    holdout_results = {}
    for h in forecast_horizons:
        print(f"\n[TEST] Forecast Horizon: {h}")
        params = best_params_dict[h]['best_params']
        hidden_dim = int(params['hidden_dim'])
        num_layers = int(params['num_layers'])
        dropout = float(params['dropout'])
        learning_rate = float(params['learning_rate'])
        batch_size = int(params['batch_size'])

        Y_df = Y_df_dict[h]
        X_shifted = shift_X_by_horizon(X_df, h)
        Y_aligned = Y_df.loc[X_shifted.index]
        X_final, Y_final = X_shifted, Y_aligned

        folds, last_val_end, _ = get_expanding_folds(X_final, Y_final, h, sequence_length, val_window_num_sequences, holdout_base)
        X_train, Y_train = X_final.iloc[:last_val_end], Y_final.iloc[:last_val_end]
        X_test, Y_test = X_final.iloc[last_val_end:], Y_final.iloc[last_val_end:]

        print(f"[INFO] Holdout Label Distribution (0s/1s): {np.bincount(Y_test.values.astype(int).flatten())}")

        scaler = StandardScaler()
        X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)
        X_test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

        X_train_seq, Y_train_seq = create_sequences(X_train_scaled, Y_train, sequence_length, h)
        X_test_seq, Y_test_seq = create_sequences(X_test_scaled, Y_test, sequence_length, h)

        X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
        Y_train_tensor = torch.tensor(Y_train_seq, dtype=torch.float32).to(device)
        X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
        Y_test_tensor = torch.tensor(Y_test_seq, dtype=torch.float32).to(device)

        model = LSTMClassifier(X_train_seq.shape[2], hidden_dim, num_layers, Y_train_seq.shape[1], dropout).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.BCEWithLogitsLoss()

        train_loader = DataLoader(TensorDataset(X_train_tensor, Y_train_tensor), batch_size=batch_size, shuffle=True)

        for epoch in range(num_epochs):
            model.train()
            for xb, yb in train_loader:
                optimizer.zero_grad()
                loss = criterion(model(xb), yb)
                loss.backward()
                optimizer.step()

        model.eval()
        preds = []
        with torch.no_grad():
            for xb in DataLoader(X_test_tensor, batch_size=batch_size):
                preds.append(torch.sigmoid(model(xb)))

        pred_tensor = torch.cat(preds, dim=0).squeeze()
        pred_bin = (pred_tensor > 0.5).int().cpu().numpy()
        y_true = Y_test_tensor.int().cpu().numpy()

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            acc = accuracy_score(y_true, pred_bin)
            f1 = f1_score(y_true, pred_bin, average='macro', zero_division=0)
            precision = precision_score(y_true, pred_bin, average='macro', zero_division=0)
            recall = recall_score(y_true, pred_bin, average='macro', zero_division=0)

        holdout_results[h] = {
            "accuracy": acc,
            "f1": f1,
            "precision": precision,
            "recall": recall
        }
        print(f"[RESULT] Horizon {h}: Accuracy = {acc:.4f}, F1 = {f1:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}")

    return holdout_results

In [32]:
holdout_results = evaluate_on_holdout(X_df, Y_df_dict, results)


[TEST] Forecast Horizon: 1
[INFO] Generated 8 folds for forecast horizon 1
[INFO] Holdout Label Distribution (0s/1s): [3620 2728]
[RESULT] Horizon 1: Accuracy = 0.6092, F1 = 0.4465, Precision = 0.4627, Recall = 0.4816

[TEST] Forecast Horizon: 5
[INFO] Generated 8 folds for forecast horizon 5
[INFO] Holdout Label Distribution (0s/1s): [2856 3252]
[RESULT] Horizon 5: Accuracy = 0.5984, F1 = 0.5961, Precision = 0.6124, Recall = 0.6073

[TEST] Forecast Horizon: 21
[INFO] Generated 8 folds for forecast horizon 21
[INFO] Holdout Label Distribution (0s/1s): [2072 3076]
[RESULT] Horizon 21: Accuracy = 0.9010, F1 = 0.4740, Precision = 0.4505, Recall = 0.5000

[TEST] Forecast Horizon: 63
[INFO] Generated 7 folds for forecast horizon 63
[INFO] Holdout Label Distribution (0s/1s): [1979 4045]
[RESULT] Horizon 63: Accuracy = 0.6493, F1 = 0.6218, Precision = 0.6311, Recall = 0.6641

[TEST] Forecast Horizon: 252
[INFO] Generated 5 folds for forecast horizon 252
[INFO] Holdout Label Distribution (0s/